Este script produce el siguiente flujo de trabajo:

1) Importar shapes desde directorio raiz
2) Importar una tabla de equivalencia de codigos
3) Crear una nueva columna USOS_AG y asignamos las categorias simplificadas.
4) Creamos una nueva columna usos_cod_a y asignamos un código numérico a cada columna
5) Eliminamos las categorias sin edificar y otros
6) Exportar los nuevos shapefile
7) A partir de los nuevos datos se calculó los porcentajes de cada uso del suelo luego sin considerar sin edificar y otros.

In [2]:
import pandas as pd
import geopandas as gpd
import os
import glob
import string



Progress bar youtube video: https://www.youtube.com/watch?v=n4E7of9BINo&t=6s&ab_channel=RobMulla

In [5]:
from tqdm import tqdm, trange ### Este es un paquete que sirve para poder hacer la barra de progreso.
from time import sleep
from tqdm.notebook import tqdm

In [3]:
#path =(r"C:/projectos/shapes/marcos_clasificacion/")
path = (r"C:/ciudades clasificacion/marcos_clasificacion_ag/")
# Change the directory
os.chdir(path)

In [10]:
dict_shapes = {}
pbar = tqdm(total = 72)
# Iterate over all the files in the directory
for file in os.listdir():
   if file.endswith('.shp') and file[0].isalpha():
       temp = gpd.read_file(file)
       dict_shapes[file] = temp
       pbar.update(1)
       sleep(0.0001)
pbar.close()

ImportError: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html

In [58]:
dict_shapes

{'Avila.shp':          COORX      COORY  AREA          REFCAT  A_TOT_EDIF  SUP_ED_0  \
 0    356146.04 4501680.61   115  6219012UL5061N         148        83   
 1    355869.04 4501678.01   110  6019701UL5051N         336         0   
 2    352397.49 4494133.66   117  2544406UK5924S         238        91   
 3    352318.21 4494121.85   135  2443220UK5924S         270       135   
 4    355938.71 4501646.15   112  6019403UL5061N         545       100   
 ...        ...        ...   ...             ...         ...       ...   
 8948 355822.73 4502136.02    56  5924007UL5052S         112        56   
 8949 355822.74 4502149.19   128  5924006UL5052S         128       128   
 8950 355818.86 4502160.66   123  5924005UL5052S         198       106   
 8951 356071.20 4499767.35  2798  6200107UL5060S           0         0   
 8952 356107.63 4499763.54  2798  6200106UL5060S           0         0   
 
       SUP_ED_1  ANTIGUEDAD  N_PLANTAS  NUM_INM_R  ... IND  OFI  COM  \
 0           65        19

In [11]:
path =(r"C:/projectos/shapes/marcos_clasificacion_ag/")
# Change the directory
os.chdir(path)

FileNotFoundError: [WinError 3] El sistema no puede encontrar la ruta especificada: 'C:/projectos/shapes/marcos_clasificacion_ag/'

In [12]:
dict_shapes = {}
pbar = tqdm(total = 72)
# Iterate over all the files in the directory
for file in os.listdir():
   if file.endswith('.shp') and file[0].isalpha():
       temp = gpd.read_file(file)
       dict_shapes[file] = temp
       pbar.update(1)
       sleep(0.0001)
pbar.close()

Exception ignored in: <function tqdm.__del__ at 0x000001EFBE5A2700>
Traceback (most recent call last):
  File "c:\Users\CRISTIAN\anaconda3\envs\spatial_uah\Lib\site-packages\tqdm\std.py", line 1145, in __del__
    self.close()
  File "c:\Users\CRISTIAN\anaconda3\envs\spatial_uah\Lib\site-packages\tqdm\notebook.py", line 283, in close
    self.disp(bar_style='danger', check_delay=False)
    ^^^^^^^^^
AttributeError: 'tqdm_notebook' object has no attribute 'disp'
Exception ignored in: <function tqdm.__del__ at 0x000001EFBE5A2700>
Traceback (most recent call last):
  File "c:\Users\CRISTIAN\anaconda3\envs\spatial_uah\Lib\site-packages\tqdm\std.py", line 1145, in __del__
    self.close()
  File "c:\Users\CRISTIAN\anaconda3\envs\spatial_uah\Lib\site-packages\tqdm\notebook.py", line 283, in close
    self.disp(bar_style='danger', check_delay=False)
    ^^^^^^^^^
AttributeError: 'tqdm_notebook' object has no attribute 'disp'
Exception ignored in: <function tqdm.__del__ at 0x000001EFBE5A2700>


ImportError: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html

In [59]:
for keys, values in dict_shapes.items():
    print(keys)

Avila.shp
Madrid.shp
Malaga.shp
Murcia.shp
Sevilla.shp


In [ ]:
# Calculo de las Áreas 

In [7]:
dict_areas = {}
for key, value in dict_shapes.items():
    temps = dict_shapes[key].copy()
    temps['Area'] = temps.area
    dict_areas[key] = temps
    

In [8]:
dict_areas["Madrid.shp"].head(2)

,COORX,COORY,AREA,REFCAT,A_TOT_EDIF,SUP_ED_0,SUP_ED_1,ANTIGUEDAD,N_PLANTAS,NUM_INM_R,...,OFI,COM,EQUIP_SANI,EQUIP_EDU,EQUIP_OTR,OCIO_ESP,HOS_REST,ED_SING,geometry,Area
0,446416.80,4472079.07,110108,6522301VK4762D,42741,39542,1407,1979,2,0,...,0,0,0,0,99,0,1,0,"POLYGON ((446311.187 4472177.764, 446312.670 4...",110108.208172
1,429373.98,4481071.39,3084,9512514VK2891D,1260,605,311,1997,1,1,...,0,0,0,0,7,0,0,0,"POLYGON ((429358.628 4481106.158, 429363.568 4...",3084.096567


In [9]:
def uso_agregado(data_in, clases):
    data_out = {}
    for key, value in data_in.items():
        data_out[key] = value.merge(clases, on = 'USO', how = 'left')
    return data_out

In [10]:
pd.options.display.float_format = "{:.2f}".format

In [11]:
clases_agregadas = pd.read_csv("C:/projectos/shapes/clases_agregadas.csv")
clases_agregadas

,USO,USO_AG
0,COM,COM
1,ED_SING,ED_SING
2,EQUIP_EDU,EQUIP
3,EQUIP_OTR,EQUIP
4,EQUIP_SANI,EQUIP
5,HOS_REST,OCIO
6,IND,IND
7,IND_MX,IND
8,OCIO_ESP,OCIO
9,OFI,OFI


In [12]:
uso_agregado_g = uso_agregado(dict_areas,clases_agregadas)

In [13]:
uso_agregado_g["Madrid.shp"]

,COORX,COORY,AREA,REFCAT,A_TOT_EDIF,SUP_ED_0,SUP_ED_1,ANTIGUEDAD,N_PLANTAS,NUM_INM_R,...,COM,EQUIP_SANI,EQUIP_EDU,EQUIP_OTR,OCIO_ESP,HOS_REST,ED_SING,geometry,Area,USO_AG
0,446416.80,4472079.07,110108,6522301VK4762D,42741,39542,1407,1979,2,0,...,0,0,0,99,0,1,0,"POLYGON ((446311.187 4472177.764, 446312.670 4...",110108.21,EQUIP
1,429373.98,4481071.39,3084,9512514VK2891D,1260,605,311,1997,1,1,...,0,0,0,7,0,0,0,"POLYGON ((429358.628 4481106.158, 429363.568 4...",3084.10,RES_UNI
2,442963.08,4465900.01,61671,3061701VK4636A,0,0,0,0,0,0,...,0,0,0,0,0,0,0,"POLYGON ((442895.447 4466121.871, 442896.697 4...",61670.94,SIN_EDIF
3,437630.29,4481134.45,353,7613546VK3871D,307,111,81,1983,2,1,...,0,0,0,8,0,0,0,"POLYGON ((437639.717 4481128.006, 437638.218 4...",353.04,RES_UNI
4,437634.97,4481118.90,394,7613545VK3871D,334,125,85,1983,2,1,...,0,0,0,6,0,0,0,"POLYGON ((437639.717 4481128.006, 437640.768 4...",394.21,RES_UNI
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
137759,449461.93,4480317.82,213,9604521VK4890D,231,79,60,2003,2,1,...,0,0,0,0,0,0,0,"POLYGON ((449473.762 4480329.503, 449474.064 4...",213.10,RES_UNI
137760,438976.12,4477397.85,900,9075503VK3797E,963,253,236,1973,2,0,...,0,0,100,0,0,0,0,"POLYGON ((438998.690 4477404.510, 438998.460 4...",900.40,EQUIP
137761,449949.76,4472168.22,962,9824508VK4792D,1076,964,112,1966,1,0,...,0,0,0,71,0,0,29,"POLYGON ((449940.522 4472196.090, 449960.481 4...",962.29,EQUIP
137762,449885.49,4472188.30,9015,9824506VK4792D,4218,3434,413,1964,2,0,...,0,0,0,0,0,0,0,"POLYGON ((449938.992 4472197.496, 449940.522 4...",9015.49,IND


In [73]:
# Agregaremos un valor numero a cada una de las categorias 

import numpy as np
import pandas as pd

def asignacion_codigo_categorias(data_in):
    data_out = {}
    for key, df in data_in.items():
        df = df.reset_index()
        usos = df['USO_AG'].unique()
        cod_usos = np.arange(len(usos))
        cod_map = pd.Series(cod_usos, index=usos)
        df['cod_usos_a'] = df['USO_AG'].map(cod_map)
        data_out[key] = df
    return data_out

In [74]:
uso_agregado_cat = asignacion_codigo_categorias(uso_agregado_g)

In [76]:
uso_agregado_cat["Madrid.shp"].cod_usos_a.unique()

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [81]:
df_final = pd.DataFrame()
for key, value in uso_agregado_cat.items():
    temp = uso_agregado_gtp_cat[key] 
    temp_cat = temp.groupby('USO_AG')['Area'].sum()
    temp_area = temp.groupby('USO_AG')['Area'].sum().sum()
    temp_porcentaje = (temp_cat/temp_area)*100
    df_temp = temp_porcentaje.to_frame().rename(columns = {'Area':key})
    df_final = pd.concat([df_final,df_temp], axis = 1)

In [82]:
df_final

,Avila.shp,Madrid.shp,Malaga.shp,Murcia.shp,Sevilla.shp
USO_AG,,,,,
COM,0.27,0.38,1.10,0.42,0.96
ED_SING,4.06,6.79,3.01,3.04,9.53
EQUIP,13.66,13.19,10.68,5.60,14.46
IND,9.48,6.67,10.81,9.07,15.35
OCIO,1.01,0.44,0.59,0.22,0.89
OFI,0.14,1.50,0.79,0.14,0.79
RES_PLU,11.08,15.03,12.49,6.72,15.73
RES_UNI,16.97,4.63,19.50,15.43,8.06
SIN_EDIF,43.34,51.16,41.02,57.11,34.23


In [17]:
df_final.to_csv(r'C:/projectos/shapes/ciudades shape/uso_ciudades_agregado_completo.csv')

# Ahora vamos a crear una funcion para eliminar el otros de cada uno de los shapes de forma iterativa

In [83]:
dict_filter_otros = {}
for key, value in uso_agregado_cat.items():
    ciudad = uso_agregado_cat[key].copy()
    ciudad1 = ciudad[ciudad.loc[:,'USO_AG'] != "OTROS"].copy()
    dict_filter_otros[key] = ciudad1

In [84]:
len(dict_filter_otros)

5

In [85]:
df_otros = pd.DataFrame()
for key, value in dict_filter_otros.items():
    temp = dict_filter_otros[key] 
    temp_cat = temp.groupby('USO_AG')['Area'].sum()
    temp_area = temp.groupby('USO_AG')['Area'].sum().sum()
    temp_porcentaje = (temp_cat/temp_area)*100
    df_temp = temp_porcentaje.to_frame().rename(columns = {'Area':key})
    df_otros = pd.concat([df_otros,df_temp], axis = 1)

In [86]:
df_otros

,Avila.shp,Madrid.shp,Malaga.shp,Murcia.shp,Sevilla.shp
USO_AG,,,,,
COM,0.27,0.38,1.10,0.43,0.96
ED_SING,4.06,6.81,3.01,3.11,9.53
EQUIP,13.66,13.22,10.68,5.73,14.46
IND,9.48,6.68,10.81,9.28,15.35
OCIO,1.01,0.44,0.59,0.23,0.89
OFI,0.14,1.50,0.79,0.14,0.79
RES_PLU,11.08,15.06,12.49,6.87,15.73
RES_UNI,16.97,4.64,19.50,15.78,8.06
SIN_EDIF,43.34,51.27,41.02,58.43,34.23


In [87]:
ciudad.columns

Index(['index', 'COORX', 'COORY', 'AREA', 'REFCAT', 'A_TOT_EDIF', 'SUP_ED_0',
       'SUP_ED_1', 'ANTIGUEDAD', 'N_PLANTAS', 'NUM_INM_R', 'USO', 'PROP_AE_AT',
       'RES_UNI', 'RES_PLU', 'IND', 'OFI', 'COM', 'EQUIP_SANI', 'EQUIP_EDU',
       'EQUIP_OTR', 'OCIO_ESP', 'HOS_REST', 'ED_SING', 'geometry', 'Area',
       'USO_AG', 'cod_usos_a'],
      dtype='object')

In [88]:
df_otros.to_csv(r'C:/projectos/shapes/ciudades shape/df_porcentajes_sin_otros.csv')

In [90]:
dict_filter = {}
for key, value in uso_agregado_cat.items():
    ciudad = uso_agregado_cat[key].copy()
    ciudad1 = ciudad[ciudad.loc[:,'USO_AG'] != "SIN_EDIF"].copy()
    ciudad2 = ciudad1[ciudad1.loc[:,'USO_AG'] != "OTROS"].copy()
    dict_filter[key] = ciudad2

In [91]:
 # Option 2

dict_filter1 = {}

for key, value in uso_agregado_cat.items():
    ciudad = uso_agregado_cat[key].copy()
    ciudad_filtered = ciudad.loc[(ciudad['USO_AG'] != "SIN_EDIF") & (ciudad['USO_AG'] != "OTROS"), :]
    dict_filter1[key] = ciudad_filtered

In [92]:
dict_filter['Madrid.shp'][dict_filter["Madrid.shp"].loc[:,'USO_AG'] == 'OTROS']

,index,COORX,COORY,AREA,REFCAT,A_TOT_EDIF,SUP_ED_0,SUP_ED_1,ANTIGUEDAD,N_PLANTAS,...,EQUIP_SANI,EQUIP_EDU,EQUIP_OTR,OCIO_ESP,HOS_REST,ED_SING,geometry,Area,USO_AG,cod_usos_a


In [93]:
for keys, values in dict_filter.items():
    dict_filter[keys].to_file('C:/projectos/shapes/ciudades_shape_ag/'+ keys)

## Vamos a calcular los porcentajes nuevamente sin otros y sin edificar ##

In [55]:
df_final_filter = pd.DataFrame()
for key, value in dict_filter.items():
    temp = dict_filter[key] 
    temp_cat = temp.groupby('USO_AG')['Area'].sum()
    temp_area = temp.groupby('USO_AG')['Area'].sum().sum()
    temp_porcentaje = (temp_cat/temp_area)*100
    df_temp = temp_porcentaje.to_frame().rename(columns = {'Area':key})
    df_final_filter = pd.concat([df_final_filter,df_temp], axis = 1)

In [56]:
df_final_filter.to_csv('C:/projectos/shapes/ciudades shape/df_final_filter.csv')